## install dependencies

In [2]:
%%capture
!pip install openai
!pip install tiktoken
!pip install chromadb
!pip install langchain
!pip install pypdf
!pip install evaluate
!pip install bert_score
!pip install rouge_score
!pip install detoxify
!pip install presidio-analyzer
!pip install Dbias
!pip install sentence_transformers

## Importing Libraries

In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import openai
from google.colab import userdata
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.llms import OpenAI
import evaluate
from detoxify import Detoxify
from presidio_analyzer import AnalyzerEngine
from presidio_analyzer.nlp_engine import NlpArtifacts
import re
import os
from textblob import TextBlob
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from Dbias.bias_classification import classifier
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import pairwise_cos_sim
from sklearn.metrics.pairwise import cosine_similarity
# Ensure you've downloaded the vader_lexicon
nltk.download('vader_lexicon')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/268M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at d4data/bias-detection-model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

## Loading files from webpage via link or local pdf

In [7]:
def loader_func():
  # from langchain_community.document_loaders import WebBaseLoader
  # loader = WebBaseLoader('link')
  # data = loader.load()
  # Load PDF
  loaders = [
      PyPDFLoader("/content/drive/MyDrive/RAI_1/02. Classification of Botany and use of Plants author Instytut Zootechniki PIB.pdf"),
      PyPDFLoader("/content/drive/MyDrive/RAI_1/06. Introduction to Botany (Syllabus) author UW Staff Web Server.pdf"),
      PyPDFLoader("/content/drive/MyDrive/RAI_1/09. Botany Basics  author Biolympiads  Your guide to biology olympiads.pdf"),
  ]
  docs = []
  for loader in loaders:
      docs.extend(loader.load())
  return docs



In [ ]:
docs = loader_func()

## Chunk spliting

In [ ]:
def spliter_func(docs):
  # Split
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size = 1500,
      chunk_overlap = 150
  )
  return text_splitter.split_documents(docs)

In [ ]:
splits = spliter_func(docs)

In [ ]:
len(splits)

380

## Importing openai Key

In [8]:
key_value = userdata.get('KeySat')
openai.api_key = key_value

## Generate response using openai library

In [9]:
def generate_response(messages, temperature = 0):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=temperature,
        top_p=0.95)
    return response.choices[0].message.content

## Defining path where vectordb would be created

In [10]:
persist_directory = '/content/drive/MyDrive/docs/chroma2/'

## Defining embedding to be used

In [11]:
embedding = OpenAIEmbeddings(openai_api_key=key_value)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
!rm -rf ./docs/chroma2

## Creating vectordb instance

In [ ]:
vectordb = Chroma.from_documents(
    documents=splits,
    persist_directory=persist_directory,
    embedding=embedding
)



## If DB already exists calling it

In [12]:
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [13]:
print(vectordb._collection.count())

380


## Defining llm to be used in langchain retrieval

In [14]:
llm = OpenAI(api_key=key_value, temperature = 0)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


## Using langchain retrieval - base similarity, compression retrieval, mmr retrieval(used for now rest are commented, uncomment them and try them) and compression+mmr retrieval... Use retrievial and chunking methods as per the requirement... Two more retrieval are sentence-window and auto-merging... this both can also be used but not written here.

In [15]:
def context_func(question,k=4, fetch_k=9):

  context = vectordb.max_marginal_relevance_search(question,k=4, fetch_k=9)


  # context = vectordb.similarity_search(question,k=4)

  # compressor = LLMChainExtractor.from_llm(llm=llm)

  # compression_retriever = ContextualCompressionRetriever(
  #     base_compressor=compressor,
  #     base_retriever=vectordb.as_retriever()
  # )
  # compressed_context = compression_retriever.get_relevant_documents(question)

  # compression_retriever1 = ContextualCompressionRetriever(
  #     base_compressor=compressor,
  #     base_retriever=vectordb.as_retriever(search_type = "mmr")
  # )
  # compressed_context = compression_retriever1.get_relevant_documents(question)
  return context

In [ ]:
# question = "Can you explain the process of photosynthesis and its importance in the plant life cycle?"

In [ ]:
# context_docs = context_func(question)
# context = ' '.join(d.page_content for d in context_docs)
# context_sources = '\n'.join(str(d.metadata) for d in context_docs)


In [ ]:
# print(context[:400])

Chapter 1—Botany Basics  • 25
light is limited, or transports them to its 
roots or developing fruits. 
Photosynthesis occurs only in the me-
sophyll  layers of plant leaves and, in some 
instances, in mesophyll cells in the stem. 
Mesophyll cells are sandwiched between 
the leaf’s upper and lower epidermis (Fig -
ure 12) and contain numerous chloroplasts , 
where photosynthesis takes place. Chlor


In [ ]:
# print(context_sources)

{'page': 22, 'source': '/content/drive/MyDrive/RAI_1/09. Botany Basics  author Biolympiads  Your guide to biology olympiads.pdf'}
{'page': 52, 'source': '/content/drive/MyDrive/RAI_1/06. Introduction to Botany (Syllabus) author UW Staff Web Server.pdf'}
{'page': 84, 'source': '/content/drive/MyDrive/RAI_1/06. Introduction to Botany (Syllabus) author UW Staff Web Server.pdf'}
{'page': 0, 'source': '/content/drive/MyDrive/RAI_1/09. Botany Basics  author Biolympiads  Your guide to biology olympiads.pdf'}


## Generating response using generate response function along with prompt

In [16]:
def generate_response_func(context, question):
  prompt = [{"role": "system", "content": """A user will give you a context and a question.
          Answer the question only based on the facts in the given context.
          Ensure that the answer is relevant to the question.
          If the question cannot be answered correctly using the context then say 'Sorry, no relevant context provided'.
          If no context is provided then answer the question without using context.
          Avoid sensitive / controversial subjects.
          Don't share confidential information."""}, \
      {"role": "user", "content": "context: " + str(context) + " question: " + str(question)}]
  response = generate_response(prompt)
  return response
# response = generate_response_func(context, question)
# print("ANSWER: \"%s\"" % response)

# RAG Traid/ Rag Evaluation

## Answer_Relevance

In [17]:
def answer_relevance_func(response, question):
  prompt = [{"role": "system", "content": """A user will give you a response and a question \
              and your task is to rate the how relevant is response\
              provided to the question provided. Your answer must be only \
              a number between 0.0 and 1.0 rounded to the nearest two \
              decimal places where 0.0 represents no relevancy of response\
              provided to the question provided and \
              1.0 represents perfect relevancy of response\
              provided to the question provided and no other text or number"""},\
      {"role": "user", "content": "Response: " + str(response) + " Question: "+ str(question)}]
  answer_relevance = generate_response(prompt)
  return answer_relevance

## Context_Relevance

In [18]:
def context_relevance_func(context, question):
  prompt = [{"role": "system", "content": """A user will give you a context and a question \
              and your task is to rate the how relevant is context\
              provided to the question provided. Your answer must be only \
              a number between 0.0 and 1.0 rounded to the nearest two \
              decimal places where 0.0 represents no relevancy and \
              1.0 represents perfect relevancy and no other text or number"""},\
      {"role": "user", "content": "Context: " + str(context) + " Question: "+ str(question)}]
  context_relevance = generate_response(prompt)
  return context_relevance

## Groundness

In [19]:
def groundness_func(context, response):
  prompt = [{"role": "system", "content": """A user will give you a context and a response \
              and your task is to rate the how much is response provided supported by context\
              provided. Your answer must be only \
              a number between 0.0 and 1.0 rounded to the nearest two \
              decimal places where 0.0 represents response provided is not at all supported by context provided and \
              1.0 represents response provided is fully supported by context provided and no other text or number"""},\
      {"role": "user", "content": "Context: " + str(context) + " Response: "+ str(response)}]
  groundness = generate_response(prompt)
  return groundness

In [22]:
# answer_relevance_score = answer_relevance_func(response, question)
# context_relevance_score = context_relevance_func(question, context)
# groundness_score = groundness_func(response, context)
# print("Answer_relevance: \"%s\"" % answer_relevance_score,"\nGroundness: \"%s\"" % groundness_score,"\nContext_relevance: \"%s\"" % context_relevance_score)

# Prompt-Response Relevance
## BLeu SCore

In [23]:
def bleu_score_func(response):
  bleu = evaluate.load("bleu")
  bleu_score = bleu.compute(predictions=[question],
              references=[response],
              max_order=2)
  return bleu_score
# bleu_score = bleu_score_func(response)
# print(bleu_score)

# Prompt-Response Relevance
## Bert Score

In [24]:
def bert_score_func(response):
  bert = evaluate.load("bertscore")
  bert_score = bert.compute(
      predictions=[question],
      references=[response],
      model_type="distilbert-base-uncased")
  return bert_score
# bert_score = bert_score_func(response)
# print(bert_score)

# Prompt-Response Relevance
## Meteor Score

In [25]:
def meteor_score_func(response):
  meteor = evaluate.load('meteor')
  meteor_score = meteor.compute(predictions=[question], references=[response])
  return meteor_score
# meteor_score = meteor_score_func(response)
# print(meteor_score)

# Prompt-Response Relevance
## Rouge Score

In [26]:
def rouge_score_func(response):
  rouge = evaluate.load('rouge')
  rouge_score = rouge.compute(predictions=[question],
                          references=[response])
  return rouge_score
# rouge_score = rouge_score_func(response)
# print(rouge_score)

# Prompt-Response Relevance
## Cosine similarity score

In [27]:
def cosine_similartiy_func(question, response):
  model = SentenceTransformer('all-MiniLM-L6-v2')
  query_embedding = model.encode([question])
  response_embedding = model.encode([response])
  cosine_dist = cosine_similarity(query_embedding, response_embedding)
  return cosine_dist[0][0]
# cosine_similartiy_score = cosine_similartiy_func(question, response)
# print(cosine_similartiy_score)

## Using openai moderation library for getting scores and see whether its flagged or not

In [28]:
def moderation_func(response):
  a = []
  a.append(response)
  moderation_score = openai.moderations.create(
      input = a
  )
  moderation_output = moderation_score.results[0]
  return moderation_output
# moderation_score = moderation_func(response)
# print(moderation_score)

## Getting toxcity scores for different type of toxicity

In [29]:
def calculate_toxicity_func(text):
    # Load the pre-trained model
    model = Detoxify('original')

    # Use the model to predict the toxicity of the text
    results = model.predict(text)

    return results
# toxicity_score = calculate_toxicity_func(response)
# print(toxicity_score)

## Data Leakage/ Privacy using regex and library

In [47]:
def detect_pii(response):
    # pii_data = {}

    # # Detect email addresses
    # email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    # emails = re.findall(email_pattern, text)
    # pii_data['Emails'] = emails

    # # Detect Indian phone numbers
    # phone_pattern = r'\b((0|\+91)?[6-9][0-9]{9})\b'
    # phone_numbers = re.findall(phone_pattern, text)
    # pii_data['Phone Numbers'] = [p[0] for p in phone_numbers]

    # # Detect Indian Aadhaar numbers
    # aadhaar_pattern = r'\b[2-9]{1}[0-9]{11}\b'
    # aadhaars = re.findall(aadhaar_pattern, text)
    # pii_data['Aadhaar Numbers'] = aadhaars
    # response = "I am Rama .a no. is FTPG8938K 9452586235 228102224617"
    engine = AnalyzerEngine()

    # Call the analyze method over your text
    results = engine.analyze(correlation_id=0,
                            text=response,
                            entities=[],
                            language='en',
                            nlp_artifacts=NlpArtifacts([], [], [], [], response, []))
    result = []
    for i in results:
      result.append("Type: " + str(i.entity_type) + ", Score: " + str(i.score))
    return result
# pii_response = detect_pii(response)
# print(pii_response)


In [33]:
def detect_pii_1(response):
    pii_data = {}
    # Detect email addresses
    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    emails = re.findall(email_pattern, response)
    pii_data['Emails'] = emails

    # Detect Indian phone numbers
    phone_pattern = r'\b((0|\+91)?[6-9][0-9]{9})\b'
    phone_numbers = re.findall(phone_pattern, response)
    pii_data['Phone Numbers'] = [p[0] for p in phone_numbers]

    # Detect Indian Aadhaar numbers
    aadhaar_pattern = r'\b[2-9]{1}[0-9]{11}\b'
    aadhaars = re.findall(aadhaar_pattern, response)
    pii_data['Aadhaar Numbers'] = aadhaars
    print(pii_data)

    engine = AnalyzerEngine()

    # Call the analyze method over your text
    results = engine.analyze(correlation_id=0,
                            text=response,
                            entities=[],
                            language='en',
                            nlp_artifacts=NlpArtifacts([], [], [], [], response, []))
    result = []
    for i in results:
      result.append("Type: " + str(i.entity_type) + ", Score: " + str(i.score))
    return result
response = "I am Rama .a no. is FTPG8938K 9452586235 228102224617"
pii_response = detect_pii_1(response)
print(pii_response)


{'Emails': [], 'Phone Numbers': ['9452586235'], 'Aadhaar Numbers': ['228102224617']}
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
['Type: IN_AADHAAR, Score: 1.0', 'Type: PHONE_NUMBER, Score: 0.4', 'Type: US_BANK_NUMBER, Score: 0.05', 'Type: US_BANK_NUMBER, Score: 0.05', 'Type: US_DRIVER_LICENSE, Score: 0.01', 'Type: US_DRIVER_LICENSE, Score: 0.01']


## Neutrality, Subjectivity and polarity score

In [34]:
def calculate_scores(text):
    blob = TextBlob(text)
    sia = SentimentIntensityAnalyzer()

    polarity = blob.sentiment.polarity
    subjectivity = blob.sentiment.subjectivity

    # VADER sentiment gives us the negativity, neutrality and positivity
    sentiment_scores = sia.polarity_scores(text)
    neutrality = sentiment_scores['neu']

    return {
        'neutrality': neutrality,
        'subjectivity': subjectivity,
        'polarity': polarity
    }
# neutrality_polarity_scores = calculate_scores(response)
# print(neutrality_polarity_scores)

## Bias Score

In [35]:
def bias_score_func(response):
  return classifier(response)
# bias_score = bias_score_func(response)
# print(f"Bias score: {bias_score}")

## Paraphrasing question and generating 3 different responses for evalution

In [37]:
def respone_gen_func(question):
  r = []
  question1 = ""
  context1 = ""
  prompt1 = [{"role": "system", "content": """A user will give you a context and a question.
          Answer the question only based on the facts in the given context.
          Ensure that the answer is relevant to the question.
          If the question cannot be answered correctly using the context then say 'Sorry, no relevant context provided'.
          If no context is provided then answer the question without using context.
          Avoid sensitive / controversial subjects.
          Don't share confidential information."""}, \
      {"role": "user", "content": "context: " + str(context1) + " question: " + str(question)}]
  prompt2 = [{"role": "system", "content": """A user will give you a query, context and a question.
          Paraphase the query in a way that meaning doesn't change
          Answer should be only the paraphased query nothing else"""}, \
      {"role": "user", "content": "Query: " + str(question)}]
  for i in range(2):
    question1 = generate_response(prompt2, 0.2)
    # print(question1)
    context1_docs = context_func(question1)
    context1 = ' '.join(d.page_content for d in context_docs)
    r.append(generate_response(prompt1))
  # for i in r:
  #   print(i)
  text = {}
  text["response1"] = response
  text["response2"] = r[0]
  text["response3"] = r[1]
  return text
# responses = respone_gen_func(question)
# print(responses)

# Response self similarity
## Cosine similarity

In [39]:
def sentence_embedding_selfsimilarity(responses):
  model = SentenceTransformer('all-MiniLM-L6-v2')
  response_embeddings = model.encode([responses["response1"]])
  response2_embeddings = model.encode([responses["response2"]])
  response3_embeddings = model.encode([responses["response3"]])

  cos_sim_with_response2 = cosine_similarity(
    response_embeddings, response2_embeddings
    )
  cos_sim_with_response3  = cosine_similarity(
    response_embeddings, response3_embeddings
    )
  cos_sim_score = (cos_sim_with_response2 + cos_sim_with_response3) / 2
  return cos_sim_score[0][0]
# sentence_embedding_selfsimilarity_score = sentence_embedding_selfsimilarity(responses)
# print(sentence_embedding_selfsimilarity_score)

#  Response self similarity
## LLM self similarity

In [40]:
def llm_selfsimilarity(responses):
        prompt3=[{
            "role": "system",
            "content": f"""You will be provided with a text passage \
            and your task is to rate the consistency of that text to \
            that of the provided context. Your answer must be only \
            a number between 0.0 and 1.0 rounded to the nearest two \
            decimal places where 0.0 represents no consistency and \
            1.0 represents perfect consistency and similarity. \n\n \
            Text passage: {responses["response1"]}. \n\n \
            Context: {responses["response2"]} \n\n \
            {responses["response3"]}."""
        }]
        return generate_response(prompt3)

# llm_selfsimilarity_score = llm_selfsimilarity(responses)
# print(llm_selfsimilarity_score)

In [41]:
question = "Can you explain the process of photosynthesis and its importance in the plant life cycle?"
context_docs = context_func(question)
context = ' '.join(d.page_content for d in context_docs)
context_sources = '\n'.join(str(d.metadata) for d in context_docs)
response = generate_response_func(context, question)
answer_relevance_score = answer_relevance_func(response, question)
context_relevance_score = context_relevance_func(question, context)
groundness_score = groundness_func(response, context)
bleu_score = bleu_score_func(response)
bert_score = bert_score_func(response)
meteor_score = meteor_score_func(response)
rouge_score = rouge_score_func(response)
moderation_score = moderation_func(response)
toxicity_score = calculate_toxicity_func(response)
pii_response = detect_pii(response)
neutrality_polarity_scores = calculate_scores(response)
cosine_similartiy_score = cosine_similartiy_func(question, response)
responses = respone_gen_func(question)
sentence_embedding_selfsimilarity_score = sentence_embedding_selfsimilarity(responses)
llm_selfsimilarity_score = llm_selfsimilarity(responses)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [42]:
print("question: ", question)
print("context: ", context[:300])
print("context_sources: ", context_sources)
print("response: ", response)
print("answer_relevance_score: ", answer_relevance_score)
print("context_relevance_score: ", context_relevance_score)
print("groundness_score: ", groundness_score)
print("bleu_score: ", bleu_score)
print("bert_score: ", bert_score)
print("meteor_score: ", meteor_score)
print("rouge_score: ", rouge_score)
print("moderation_score: ", moderation_score)
print("toxicity_score: ", toxicity_score)
print("pii_response: ", pii_response)
print("neutrality_polarity_scores: ", neutrality_polarity_scores)
print("cosine_similartiy_score: ", cosine_similartiy_score)
print("sentence_embedding_selfsimilarity_score: ", sentence_embedding_selfsimilarity_score)
print("llm_selfsimilarity_score : ", llm_selfsimilarity_score)

question:  Can you explain the process of photosynthesis and its importance in the plant life cycle?
context:  Chapter 1—Botany Basics  • 25
light is limited, or transports them to its 
roots or developing fruits. 
Photosynthesis occurs only in the me-
sophyll  layers of plant leaves and, in some 
instances, in mesophyll cells in the stem. 
Mesophyll cells are sandwiched between 
the leaf’s upper and lower e
context_sources:  {'page': 22, 'source': '/content/drive/MyDrive/RAI_1/09. Botany Basics  author Biolympiads  Your guide to biology olympiads.pdf'}
{'page': 52, 'source': '/content/drive/MyDrive/RAI_1/06. Introduction to Botany (Syllabus) author UW Staff Web Server.pdf'}
{'page': 84, 'source': '/content/drive/MyDrive/RAI_1/06. Introduction to Botany (Syllabus) author UW Staff Web Server.pdf'}
{'page': 0, 'source': '/content/drive/MyDrive/RAI_1/09. Botany Basics  author Biolympiads  Your guide to biology olympiads.pdf'}
response:  Photosynthesis is the process by which plants use li

## Paraphasing the question 3 times and getting all the information(Rag traid, toxicity, privacy,etc) for only for 1st question-response... Using other 3 responses for Response-self similarity... Storing it into pandas dataframe as well as logging it...

In [52]:
import pandas as pd

# Initialize an empty DataFrame
df = pd.DataFrame(columns=["Question","Context","Response", "Context_sources","Answer_relevance_score","Context_relevance_score",
                           "Groundness_score","Bleu_score","Bert_score","Cosine_similartiy_score","Meteor_score",
                           "Rouge_score","Moderation_score","Toxicity_score","PII_response","Neutrality_Polarity_scores","Baises_score",
                           "Sentence_embedding_selfsimilarity_score","Llm_selfsimilarity_score"])
path = "/content/drive/MyDrive/RAI_1/log.csv"


In [53]:
first_question = True
while True:
    if first_question:
        question = input("\nEnter your question (or type 'quit' to exit): ").strip()
    else:
        question = input("\nWhat's your next question (or type 'quit' to exit): ").strip()

    if question.lower() == "quit":
        break

    if question == "":
        continue

    first_question = False

    context_docs = context_func(question)
    context = ' '.join(d.page_content for d in context_docs)
    context_sources = '\n'.join(str(d.metadata) for d in context_docs)
    response = generate_response_func(context, question)
    answer_relevance_score = answer_relevance_func(response, question)
    context_relevance_score = context_relevance_func(question, context)
    groundness_score = groundness_func(response, context)
    bleu_score = bleu_score_func(response)
    bert_score = bert_score_func(response)
    meteor_score = meteor_score_func(response)
    rouge_score = rouge_score_func(response)
    moderation_score = moderation_func(response)
    toxicity_score = calculate_toxicity_func(response)
    pii_response = detect_pii(response)
    neutrality_polarity_scores = calculate_scores(response)
    cosine_similartiy_score = cosine_similartiy_func(question, response)
    bias_score = bias_score_func(response)
    responses = respone_gen_func(question)
    sentence_embedding_selfsimilarity_score = sentence_embedding_selfsimilarity(responses)
    llm_selfsimilarity_score = llm_selfsimilarity(responses)

    print("\nQuestion: ", question)
    print("Response: ", response)
    print("Context_sources: ", context_sources)
    df = df.append({"Question":question,"Context":context,"Response":response, "Context_sources":context_sources,
                    "Answer_relevance_score":answer_relevance_score,"Context_relevance_score":context_relevance_score,
                    "Groundness_score":groundness_score,"Bleu_score":bleu_score,"Bert_score":bert_score,
                    "Meteor_score":meteor_score,"Rouge_score":rouge_score,"Moderation_score":moderation_score,
                    "Toxicity_score":toxicity_score,"PII_response":pii_response,"Neutrality_Polarity_scores":neutrality_polarity_scores,
                    "Cosine_similartiy_score":cosine_similartiy_score,"Baises_score":bias_score,
                    "Sentence_embedding_selfsimilarity_score":sentence_embedding_selfsimilarity_score,
                    "Llm_selfsimilarity_score":llm_selfsimilarity_score}, ignore_index=True)


# Specify the file path
file_path = path

# Check if the file exists
if os.path.isfile(file_path):
    # If the file exists, append without writing the header
    df.to_csv(file_path, mode='a', header=False)
else:
    # If the file does not exist, create it and write the header
    df.to_csv(file_path, mode='w')
# df.to_csv(path, index=False)



Enter your question (or type 'quit' to exit): Can you explain the process of photosynthesis and its importance in the plant life cycle?


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
<ipython-input-53-28b98832a129>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Question":question,"Context":context,"Response":response, "Context_sources":context_sources,



Question:  Can you explain the process of photosynthesis and its importance in the plant life cycle?
Response:  Photosynthesis is the process by which plants use light energy to convert carbon dioxide and water into glucose and oxygen. This process occurs in the chloroplasts of plant cells, specifically in the mesophyll layers of plant leaves. Chlorophyll, the green pigment in chloroplasts, absorbs light energy from the sun to drive the chemical reactions of photosynthesis. The importance of photosynthesis in the plant life cycle is immense as it is the primary way plants produce their own food, which is essential for their growth and survival. Additionally, photosynthesis is crucial for producing oxygen, which is released into the atmosphere and is vital for many living organisms, including humans.
Context_sources:  {'page': 22, 'source': '/content/drive/MyDrive/RAI_1/09. Botany Basics  author Biolympiads  Your guide to biology olympiads.pdf'}
{'page': 52, 'source': '/content/drive/M

In [54]:
pd.set_option('display.max_colwidth', None)
df

,Question,Context,Response,Context_sources,Answer_relevance_score,Context_relevance_score,Groundness_score,Bleu_score,Bert_score,Cosine_similartiy_score,Meteor_score,Rouge_score,Moderation_score,Toxicity_score,PII_response,Neutrality_Polarity_scores,Baises_score,Sentence_embedding_selfsimilarity_score,Llm_selfsimilarity_score
0,Can you explain the process of photosynthesis and its importance in the plant life cycle?,"Chapter 1—Botany Basics • 25\nlight is limited, or transports them to its \nroots or developing fruits. \nPhotosynthesis occurs only in the me-\nsophyll layers of plant leaves and, in some \ninstances, in mesophyll cells in the stem. \nMesophyll cells are sandwiched between \nthe leaf’s upper and lower epidermis (Fig -\nure 12) and contain numerous chloroplasts , \nwhere photosynthesis takes place. Chloro -\nplasts are incredibly small. One square mil -\nlimeter, about the size of a period on a page, \nwould contain 400,000 chloroplasts. \nChlorophyll , the pigment that makes \nleaves green, is found in the chloroplasts. It \nis responsible for trapping light energy from \nthe sun. Chloroplasts are arranged per -\npendicular to incoming sun rays to absorb \nmaximum sunlight.\nPhotosynthesis stops if any of the ingre -\ndients for photosynthesis — light, water and \ncarbon dioxide — are lacking. If any factor \nis absent for a long period of time, a plant \nwill die. Two of these factors are described \nbelow. \nLight \nPhotosynthesis depends on the quantity, \nquality and duration of light. In general, as \nintensity increases, so does photosynthesis. \nHowever, for each plant species, there is \na maximum level of light intensity above \nwhich photosynthesis does not increase. \nMany garden crops, such as tomatoes, \nrespond best to maximum sunlight. Tomato \nproduction decreases drastically as light in -\ntensity drops, and only a few tomato variet -\nies produce any fruit under minimal sunlight What the .. Is a plant??\n(3) Lives through \nPhotosynthesis The second role of Plants on the Planet Îbiomass\nWhile photosynthetic bacteria use the light energy to \nproduce ATP (Adenosin Triphosphate – the universal \ncurrency of energy in living beings) algae and plants are \nthe major creators of biomass on the planet. Biomass \nstores a certain percentage of the solar energy and most\nheterotrophic organisms depend on it for their existence. ❂Botany Basics 1\nTopics in This chapTer ❂\nBy Ann Marie VanDerZanden, Extension Master \nGardener State Coordinator, Oregon State \nUniversity.\nAdapted by Jay Moore, former Land Resources \nAgent, Cooperative Extension Service, University \nof Alaska Fairbanks. ❂Plant life cycles\n ❂Internal plant parts\n ❂External plant parts\n ❂Plant growth and development\n ❂Environmental factors \naffecting growth\n ❂Plants in communities\n ❂Plant hormones and growth \nregulators\nTable 1.—Comparison between monocots and dicots.\nStructure Monocots Dicots\nSeed leaves One Two\n(cotyledons)\nVascular system Xylem and phloem are paired in bundles, Xylem and phloem form rings inside the stem. \n which are dispersed throughout the stem. The phloem forms an outer ring, the xylem \n an inner ring.\nFloral parts Usually in threes or multiples of three. Usually in multiples of four or five.\nLeaves Often parallel-veined. Generally net-veined.P lants are unique and essential to life on earth. Unlike \nmost living things, plants make their own food from \nsunlight and water. Either directly or indirectly, they \nare the primary food source for humans and other animals. \nAdditionally, they provide fuel, replenish the earth’s oxygen \nsupply, prevent soil erosion, slow down wind movement, \ncool the atmosphere, provide wildlife habitat, supply me -\ndicinal compounds and beautify our surroundings. \nMany plants are familiar to us, and we can identify and \nappreciate them based on their external structures. How -","Photosynthesis is the process by which plants use light energy to convert carbon dioxi

## Paraphasing the question 5 times and getting all the information(Rag traid, toxicity, privacy,etc) for all the question-response... Storing it into pandas dataframe as well as logging it... please note that Response-self similarity score is in last column because its calculated only once for all responses combined

In [55]:
def sentence_embedding_selfsimilarity1(responses):
  model = SentenceTransformer('all-MiniLM-L6-v2')
  response_embeddings = model.encode([responses[0]])
  response2_embeddings = model.encode([responses[1]])
  response3_embeddings = model.encode([responses[2]])
  response4_embeddings = model.encode([responses[3]])
  response5_embeddings = model.encode([responses[4]])

  cos_sim_with_response2 = cosine_similarity(
    response_embeddings, response2_embeddings
    )
  cos_sim_with_response3  = cosine_similarity(
    response_embeddings, response3_embeddings
    )
  cos_sim_with_response4  = cosine_similarity(
    response_embeddings, response4_embeddings
    )
  cos_sim_with_response5  = cosine_similarity(
    response_embeddings, response5_embeddings
    )
  cos_sim_score = (cos_sim_with_response2 + cos_sim_with_response3 + cos_sim_with_response4 + cos_sim_with_response5) / 4
  return cos_sim_score[0][0]
# sentence_embedding_selfsimilarity_score = sentence_embedding_selfsimilarity(responses)
# print(sentence_embedding_selfsimilarity_score)

In [56]:
def llm_selfsimilarity1(responses):
        prompt3=[{
            "role": "system",
            "content": f"""You will be provided with a text passage \
            and your task is to rate the consistency of that text to \
            that of the provided context. Your answer must be only \
            a number between 0.0 and 1.0 rounded to the nearest two \
            decimal places where 0.0 represents no consistency and \
            1.0 represents perfect consistency and similarity. \n\n \
            Text passage: {responses[0]}. \n\n \
            Context: {responses[1]} \n\n \
            {responses[2]} \n\n \
            {responses[3]} \n\n \
            {responses[4]}."""
        }]
        return generate_response(prompt3)


# llm_selfsimilarity_score = llm_selfsimilarity(responses)
# print(llm_selfsimilarity_score)

In [61]:
import pandas as pd

# Initialize an empty DataFrame
df1 = pd.DataFrame(columns=["Question","Context","Response", "Context_sources","Answer_relevance_score","Context_relevance_score",
                           "Groundness_score","Bleu_score","Bert_score","Cosine_similartiy_score","Meteor_score",
                           "Rouge_score","Moderation_score","Toxicity_score","PII_response","Neutrality_Polarity_Subjectivity_scores","Baises_score",
                           "Sentence_embedding_selfsimilarity_score","Llm_selfsimilarity_score"])
path1 = "/content/drive/MyDrive/RAI_1/log1.csv"


In [62]:
first_question = True
responses = []
while True:
    responses = []
    if first_question:
        question = input("\nEnter your question (or type 'quit' to exit): ").strip()
    else:
        question = input("\nWhat's your next question (or type 'quit' to exit): ").strip()

    if question.lower() == "quit":
        break

    if question == "":
        continue

    first_question = False

    context_docs = context_func(question)
    context = ' '.join(d.page_content for d in context_docs)
    context_sources = '\n'.join(str(d.metadata) for d in context_docs)
    response = generate_response_func(context, question)
    answer_relevance_score = answer_relevance_func(response, question)
    context_relevance_score = context_relevance_func(question, context)
    groundness_score = groundness_func(response, context)
    bleu_score = bleu_score_func(response)
    bert_score = bert_score_func(response)
    meteor_score = meteor_score_func(response)
    rouge_score = rouge_score_func(response)
    moderation_score = moderation_func(response)
    toxicity_score = calculate_toxicity_func(response)
    pii_response = detect_pii(response)
    neutrality_polarity_scores = calculate_scores(response)
    bias_score = bias_score_func(response)
    cosine_similartiy_score = cosine_similartiy_func(question, response)
    responses.append(response)
    # sentence_embedding_selfsimilarity_score = sentence_embedding_selfsimilarity(responses)
    # llm_selfsimilarity_score = llm_selfsimilarity(responses)

    print("\nQuestion: ", question)
    print("Response: ", response)
    print("Context_sources: ", context_sources)
    df1 = df1.append({"Question":question,"Context":context,"Response":response, "Context_sources":context_sources,
                    "Answer_relevance_score":answer_relevance_score,"Context_relevance_score":context_relevance_score,
                    "Groundness_score":groundness_score,"Bleu_score":bleu_score,"Bert_score":bert_score,
                    "Meteor_score":meteor_score,"Rouge_score":rouge_score,"Moderation_score":moderation_score,
                    "Toxicity_score":toxicity_score,"PII_response":pii_response,"Neutrality_Polarity_Subjectivity_scores":neutrality_polarity_scores,
                    "Baises_score": bias_score,"Cosine_similartiy_score":cosine_similartiy_score}, ignore_index=True)

    for i in range(4):
      prompt2 = [{"role": "system", "content": """A user will give you a query, context and a question.
          Paraphase the query in a way that meaning doesn't change
          Answer should be only the paraphased query nothing else"""}, \
      {"role": "user", "content": "Query: " + str(question)}]
      question1 = generate_response(prompt2, 0.2)
      context_docs = context_func(question1)
      context = ' '.join(d.page_content for d in context_docs)
      context_sources = '\n'.join(str(d.metadata) for d in context_docs)
      response = generate_response_func(context, question1)
      answer_relevance_score = answer_relevance_func(response, question1)
      context_relevance_score = context_relevance_func(question1, context)
      groundness_score = groundness_func(response, context)
      bleu_score = bleu_score_func(response)
      bert_score = bert_score_func(response)
      meteor_score = meteor_score_func(response)
      rouge_score = rouge_score_func(response)
      moderation_score = moderation_func(response)
      toxicity_score = calculate_toxicity_func(response)
      pii_response = detect_pii(response)
      neutrality_polarity_scores = calculate_scores(response)
      bias_score = bias_score_func(response)
      cosine_similartiy_score = cosine_similartiy_func(question1, response)
      responses.append(response)
      df1 = df1.append({"Question":question1,"Context":context,"Response":response, "Context_sources":context_sources,
                    "Answer_relevance_score":answer_relevance_score,"Context_relevance_score":context_relevance_score,
                    "Groundness_score":groundness_score,"Bleu_score":bleu_score,"Bert_score":bert_score,
                    "Meteor_score":meteor_score,"Rouge_score":rouge_score,"Moderation_score":moderation_score,
                    "Toxicity_score":toxicity_score,"PII_response":pii_response,"Neutrality_Polarity_Subjectivity_scores":neutrality_polarity_scores,
                    "Baises_score": bias_score,"Cosine_similartiy_score":cosine_similartiy_score}, ignore_index=True)
    # print(len(responses))
    sentence_embedding_selfsimilarity_score = sentence_embedding_selfsimilarity1(responses)
    llm_selfsimilarity_score = llm_selfsimilarity1(responses)

    df1 = df1.append({"Sentence_embedding_selfsimilarity_score":sentence_embedding_selfsimilarity_score,
                    "Llm_selfsimilarity_score":llm_selfsimilarity_score}, ignore_index=True)

# Specify the file path
file_path = path1

# Check if the file exists
if os.path.isfile(file_path):
    # If the file exists, append without writing the header
    df1.to_csv(file_path, mode='a', header=False)
else:
    # If the file does not exist, create it and write the header
    df1.to_csv(file_path, mode='w')

# df1.to_csv(path, index=False)



Enter your question (or type 'quit' to exit): Can you explain the process of photosynthesis and its importance in the plant life cycle?


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!



Question:  Can you explain the process of photosynthesis and its importance in the plant life cycle?
Response:  Photosynthesis is the process by which plants use light energy to convert carbon dioxide and water into glucose and oxygen. This process occurs in the chloroplasts of plant cells, specifically in the mesophyll layers of leaves. Chlorophyll, the green pigment in chloroplasts, absorbs light energy from the sun to drive this chemical reaction. Photosynthesis is crucial for plants as it is their primary way of producing food (glucose) and releasing oxygen into the atmosphere, which is essential for all living organisms.
Context_sources:  {'page': 22, 'source': '/content/drive/MyDrive/RAI_1/09. Botany Basics  author Biolympiads  Your guide to biology olympiads.pdf'}
{'page': 52, 'source': '/content/drive/MyDrive/RAI_1/06. Introduction to Botany (Syllabus) author UW Staff Web Server.pdf'}
{'page': 84, 'source': '/content/drive/MyDrive/RAI_1/06. Introduction to Botany (Syllabus) au

<ipython-input-62-20e0c4ca19ff>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append({"Question":question,"Context":context,"Response":response, "Context_sources":context_sources,
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
<ipython-input-62-20e0c4ca19ff>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append({"Question":question1,"Context":context,"Response":response, "Context_sources":context_sources,
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet 

5


<ipython-input-62-20e0c4ca19ff>:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append({"Sentence_embedding_selfsimilarity_score":sentence_embedding_selfsimilarity_score,



What's your next question (or type 'quit' to exit): quit


In [63]:
pd.set_option('display.max_colwidth', None)
df1

,Question,Context,Response,Context_sources,Answer_relevance_score,Context_relevance_score,Groundness_score,Bleu_score,Bert_score,Cosine_similartiy_score,Meteor_score,Rouge_score,Moderation_score,Toxicity_score,PII_response,Neutrality_Polarity_Subjectivity_scores,Baises_score,Sentence_embedding_selfsimilarity_score,Llm_selfsimilarity_score
0,Can you explain the process of photosynthesis and its importance in the plant life cycle?,"Chapter 1—Botany Basics • 25\nlight is limited, or transports them to its \nroots or developing fruits. \nPhotosynthesis occurs only in the me-\nsophyll layers of plant leaves and, in some \ninstances, in mesophyll cells in the stem. \nMesophyll cells are sandwiched between \nthe leaf’s upper and lower epidermis (Fig -\nure 12) and contain numerous chloroplasts , \nwhere photosynthesis takes place. Chloro -\nplasts are incredibly small. One square mil -\nlimeter, about the size of a period on a page, \nwould contain 400,000 chloroplasts. \nChlorophyll , the pigment that makes \nleaves green, is found in the chloroplasts. It \nis responsible for trapping light energy from \nthe sun. Chloroplasts are arranged per -\npendicular to incoming sun rays to absorb \nmaximum sunlight.\nPhotosynthesis stops if any of the ingre -\ndients for photosynthesis — light, water and \ncarbon dioxide — are lacking. If any factor \nis absent for a long period of time, a plant \nwill die. Two of these factors are described \nbelow. \nLight \nPhotosynthesis depends on the quantity, \nquality and duration of light. In general, as \nintensity increases, so does photosynthesis. \nHowever, for each plant species, there is \na maximum level of light intensity above \nwhich photosynthesis does not increase. \nMany garden crops, such as tomatoes, \nrespond best to maximum sunlight. Tomato \nproduction decreases drastically as light in -\ntensity drops, and only a few tomato variet -\nies produce any fruit under minimal sunlight What the .. Is a plant??\n(3) Lives through \nPhotosynthesis The second role of Plants on the Planet Îbiomass\nWhile photosynthetic bacteria use the light energy to \nproduce ATP (Adenosin Triphosphate – the universal \ncurrency of energy in living beings) algae and plants are \nthe major creators of biomass on the planet. Biomass \nstores a certain percentage of the solar energy and most\nheterotrophic organisms depend on it for their existence. ❂Botany Basics 1\nTopics in This chapTer ❂\nBy Ann Marie VanDerZanden, Extension Master \nGardener State Coordinator, Oregon State \nUniversity.\nAdapted by Jay Moore, former Land Resources \nAgent, Cooperative Extension Service, University \nof Alaska Fairbanks. ❂Plant life cycles\n ❂Internal plant parts\n ❂External plant parts\n ❂Plant growth and development\n ❂Environmental factors \naffecting growth\n ❂Plants in communities\n ❂Plant hormones and growth \nregulators\nTable 1.—Comparison between monocots and dicots.\nStructure Monocots Dicots\nSeed leaves One Two\n(cotyledons)\nVascular system Xylem and phloem are paired in bundles, Xylem and phloem form rings inside the stem. \n which are dispersed throughout the stem. The phloem forms an outer ring, the xylem \n an inner ring.\nFloral parts Usually in threes or multiples of three. Usually in multiples of four or five.\nLeaves Often parallel-veined. Generally net-veined.P lants are unique and essential to life on earth. Unlike \nmost living things, plants make their own food from \nsunlight and water. Either directly or indirectly, they \nare the primary food source for humans and other animals. \nAdditionally, they provide fuel, replenish the earth’s oxygen \nsupply, prevent soil erosion, slow down wind movement, \ncool the atmosphere, provide wildlife habitat, supply me -\ndicinal compounds and beautify our surroundings. \nMany plants are familiar to us, and we can identify and \nappreciate them based on their external structures. How -","Photosynthesis is the process by which plants use light energy to convert

## Ragchatbot with transparency showing response sources in output.

In [64]:
first_question = True
responses = []
while True:
    responses = []
    if first_question:
        question = input("\nEnter your question (or type 'quit' to exit): ").strip()
    else:
        question = input("\nWhat's your next question (or type 'quit' to exit): ").strip()

    if question.lower() == "quit":
        break

    if question == "":
        continue

    first_question = False

    context_docs = context_func(question)
    context = ' '.join(d.page_content for d in context_docs)
    context_sources = '\n'.join(str(d.metadata) for d in context_docs)
    response = generate_response_func(context, question)
    print("\nQuestion: ", question)
    print("Response: ", response)
    print("Context_sources: ", context_sources)


Enter your question (or type 'quit' to exit): Can you explain the process of photosynthesis and its importance in the plant life cycle?

Question:  Can you explain the process of photosynthesis and its importance in the plant life cycle?
Response:  Photosynthesis is the process by which plants use light energy to convert carbon dioxide and water into glucose and oxygen. This process occurs in the chloroplasts of plant cells, specifically in the mesophyll layers of plant leaves. Chlorophyll, the green pigment in chloroplasts, absorbs light energy from the sun to drive this chemical reaction. The importance of photosynthesis in the plant life cycle is immense as it is the primary way plants produce their own food, which is essential for their growth and survival. Additionally, photosynthesis is crucial for producing oxygen, which is released into the atmosphere and is vital for many living organisms, including humans.
Context_sources:  {'page': 22, 'source': '/content/drive/MyDrive/RAI_